In [14]:
import pandas as pd
import json

In [15]:
def save_to_jsonl(mtd, outfilename):
    with open(outfilename, 'a+', encoding='utf8') as outfile:
        #for entry in JSON_file:
        json.dump(mtd, outfile, ensure_ascii=False)
        outfile.write('\n')

In [5]:
###############
# Use this fnx to get all habitats for a given micro-organism
# Input : data -> relations.full pandas Dataframe
# Output : Jsonl file from save_to_jsonl() fnx, to be indexed in ES (logstash input)
#############
def process_beyond_db(data):
    longList = []
    D = {}
    data['orgdict']=data[['form_microorganism','taxid','pmid','paths_microorganism', 'offset_microorganism', 'form_habitat']].to_dict("records")
    data['microorganism'] = 'microorganism'
    df = data[['microorganism','orgdict']]
    df['microorganism'] = df['microorganism'].apply(str) #Converting integer column 'label' to string
    df = df.groupby('microorganism')['orgdict'].apply(list) 
    desired_dict = df.to_dict()
    
    data['habtdict']=data[['form_habitat','obtid','pmid','paths_habitat', 'offset_habitat']].to_dict("records")
    data['habitat'] = 'habitat'
    df = data[['habitat','habtdict']]
    df['habitat'] = df['habitat'].apply(str) #Converting integer column 'label' to string
    df = df.groupby('habitat')['habtdict'].apply(list) 
    desired_dict_hab = df.to_dict()
    
    d = (pd.DataFrame(desired_dict['microorganism'])
       .groupby(['form_microorganism'])
       .agg(set)
       .reset_index()
       .to_dict('r'))
    
    dhb = (pd.DataFrame(desired_dict_hab['habitat'])
       .groupby(['form_habitat'])
       .agg(set)
       .reset_index()
       .to_dict('r'))
    
    for v in d:
        v['habitats'] = []
        v['taxid'] = list(v['taxid'])[0]
    #     v['pmid'] = list(v['pmid'])
        v['pmid'] = [str(val) for val in list(v['pmid'])]
        v['paths_microorganism'] = list(v['paths_microorganism'])[0]
        v['offset_microorganism'] = list(v['offset_microorganism'])
        v['form_habitat'] = list(v['form_habitat'])
        
    for v in dhb:
    #     v['habitats'] = []
        v['obtid'] = list(v['obtid'])[0]
#         v['pmid'] = list(v['pmid'])
        v['pmid'] = [str(val) for val in list(v['pmid'])]
        v['paths_habitat'] = list(v['paths_habitat'])[0]
        v['offset_habitat'] = list(v['offset_habitat'])[0]
        
    for i in range(len(d)):
        for j in range(len(dhb)):
            if dhb[j]['form_habitat'] in d[i]['form_habitat']:
                d[i]['habitats'].append(dhb[j]) 
            #     print(dhb[0]['form_habitat'])
    
    # D['Microorganism'].append(d[0])
    for mi in d:
#         D = {}
        D['Microorganism'] = []
        D['Microorganism'].append(mi)
#         save_to_jsonl(D, 'beyond_db_for_nested_test.jsonl')
#         longList.append(D)
#     return longList
    return D

In [6]:
file = "/home/rodrique/Bureau/Jupyter-notebook/Beyond/data-examples_v1/relations.full.tsv/relations_min.tsv"
df = pd.read_csv(file, sep = '\t')

<IPython.core.display.Javascript object>

In [7]:
df_min = df[:100]

In [8]:
df_min.columns

Index(['batch', 'pmid', 'section', 'taxid', 'form_microorganism',
       'lemma_microorganism', 'canonical_microorganism', 'paths_microorganism',
       'obtid', 'form_habitat', 'lemma_habitat', 'canonical_habitat',
       'paths_habitat', 'offset_microorganism', 'offset_habitat'],
      dtype='object')

In [20]:
# df_min = df_min[['taxid', 'form_microorganism',
#        'obtid', 'form_habitat']]

In [9]:
df_min.head()

,batch,pmid,section,taxid,form_microorganism,lemma_microorganism,canonical_microorganism,paths_microorganism,obtid,form_habitat,lemma_habitat,canonical_habitat,paths_habitat,offset_microorganism,offset_habitat
0,0,7811023.0,abstract,ncbi:5722,Trichomonas vaginalis,Trichomonas_vaginalis,Trichomonas vaginalis,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:2611341/ncb...,OBT:003220,AIDS patients,AIDS patient,patient,/OBT:000001/OBT:000010/OBT:000089/OBT:000193/O...,266-287,335-348
1,0,7811023.0,abstract,ncbi:5741,Giardia lamblia,Giardia_lamblia,Giardia intestinalis,/ncbi:1/ncbi:131567/ncbi:2759/ncbi:2611341/ncb...,OBT:003220,AIDS patients,AIDS patient,patient,/OBT:000001/OBT:000010/OBT:000089/OBT:000193/O...,292-307,335-348
2,0,23481468.0,abstract,ncbi:562,Escherichia coli,Escherichia_coli,Escherichia coli,/ncbi:1/ncbi:131567/ncbi:2/ncbi:1224/ncbi:1236...,OBT:000062,mammalian cell culture,mammalian cell culture,cell culture,/OBT:000001/OBT:000007/OBT:000062,361-377,379-401
3,0,23481468.0,abstract,ncbi:562,Escherichia coli,Escherichia_coli,Escherichia coli,/ncbi:1/ncbi:131567/ncbi:2/ncbi:1224/ncbi:1236...,OBT:000061,mammalian cell,mammalian cell,cell,/OBT:000001/OBT:000014/OBT:000061,361-377,379-393
4,0,23481468.0,abstract,ncbi:562,Escherichia coli,Escherichia_coli,Escherichia coli,/ncbi:1/ncbi:131567/ncbi:2/ncbi:1224/ncbi:1236...,OBT:001625,mammalian,mammalian,mammalian,/OBT:000001/OBT:000010/OBT:000089/OBT:000193/O...,361-377,379-388


In [11]:
D = process_beyond_db(df_min)
D

/home/rodrique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/rodrique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/rodrique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

<IPython.core.display.Javascript object>

/home/rodrique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.


<IPython.core.display.Javascript object>

/home/rodrique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.


{'Microorganism': [{'form_microorganism': 'viruses',
   'taxid': 'ncbi:10239',
   'pmid': ['28688828.0'],
   'paths_microorganism': '/ncbi:1/ncbi:10239',
   'offset_microorganism': ['11-18'],
   'form_habitat': ['community clinics', 'patients with diarrheal'],
   'habitats': [{'form_habitat': 'community clinics',
     'obtid': 'OBT:002694',
     'pmid': ['28688828.0'],
     'paths_habitat': '/OBT:000001/OBT:000006/OBT:000066/OBT:000403/OBT:000963/OBT:002518/OBT:002694,/OBT:000001/OBT:000011/OBT:002518/OBT:002694',
     'offset_habitat': '84-101'},
    {'form_habitat': 'patients with diarrheal',
     'obtid': 'OBT:003220',
     'pmid': ['28688828.0'],
     'paths_habitat': '/OBT:000001/OBT:000010/OBT:000089/OBT:000193/OBT:000491/OBT:003156/OBT:003220,/OBT:000001/OBT:000010/OBT:000089/OBT:000193/OBT:000946/OBT:001374/OBT:001625/OBT:002201/OBT:002488/OBT:003156/OBT:003220',
     'offset_habitat': '56-79'}]}]}

In [9]:
###### 
#  Use this fnx to get each relation (miroorganism <-> habitats) with translated paths
# Build Losgstash input data for ES from these input data :
# 1 - relations file
# 2 - BioNLP_onto,
# 3 - taxid_microorganisms
#####
def beyond_db_preprocessv1(relations,BioNLP_onto,taxid_microorganisms):
#     relations = "/home/rodrique/Bureau/Jupyter-notebook/Beyond/data-examples/relations.full.tsv/relations_min.tsv"
    data = pd.read_csv(relations, delimiter='\t')
    df = data[['form_microorganism', 'lemma_microorganism', 'paths_microorganism','pmid', 'section', 'taxid',
        'canonical_microorganism', 
       'obtid', 'form_habitat', 'lemma_habitat', 'canonical_habitat',
       'paths_habitat', 'offset_microorganism', 'offset_habitat']]
    filtered_df = df[df['form_microorganism'].notnull()]
    
    filtered_df_dic = filtered_df.to_dict('records')
    val = list(map(dict, set(tuple(sorted(d.items())) for d in filtered_df_dic)))
    print(len(val))
    
    test = filtered_df[:5] # consider subset for test
    
    # define translated path --> ordered list of habitats
    test['translated_path_habitat'] = test['paths_habitat'].str.split("/")
        
    dc = {}
    synod = {}
    with open (BioNLP_onto, "r") as hfile:
        sp = hfile.read()
        lines = sp.split("\n\n")
        for line in lines:
            item = line.split('\n\n\n')
            items = [i.split('\n') for i in item]
            items = [x.split(': ') for x in items[0]]
            id_name = [l for l in items if l[0]=='id' or l[0]=='name']
            syno =  [l for l in items if l[0]=='id' or l[0]=='synonym']

            if len(id_name)>0:
                dc[id_name[0][1]] = id_name[1][1]
            #   print(syno)
            ########## We can add synonyms if needed
            if len(syno)>1:
                synod[syno[0][1]] = [ i[1] for i in syno[1:]]
            ############

    for k,v in dc.items():
        test['translated_path_habitat'] = test.apply(lambda x:[i.replace(k,v) for i in x['translated_path_habitat'] if i], axis=1)
        
    # pop current habitat from the path
    test['translated_path_habitat'] = test['translated_path_habitat'].apply(lambda x:x[:-1])
    
    
    ############Build translated path for each micro-organism
#     taxid_microorganisms = '/home/rodrique/Bureau/Jupyter-notebook/Beyond/data-examples/taxid_microorganisms.tsv/taxid_microorganisms.tsv'
    taxid_data = pd.read_csv(taxid_microorganisms, delimiter='\t')
    
    taxid_data = taxid_data[ ['taxid', 'taxon']]
    
    taxid_data_dic = taxid_data.set_index('taxid').T.to_dict('r')
    
    test['translated_paths_microorganism'] = test['paths_microorganism'].str.split("/")
    
    for k,v in taxid_data_dic[0].items():
        test['translated_paths_microorganism'] = test.apply(lambda x:[i.replace(k,v) for i in x['translated_paths_microorganism'] if i], axis=1)
    
    # pop current habitat from the path
    test['translated_paths_microorganism'] = test['translated_paths_microorganism'].apply(lambda x:x[:-1])

    test_dic = test.to_dict('records')    
    for d in test_dic:
        d['pmid'] = str(d['pmid'])
        
    ################### Save record!
    for mtd in test_dic:
        save_to_jsonl(mtd, 'beyond_db_uniq5.jsonl')

In [ ]:
###### 
#  Use this fnx to get each relation (miroorganism <-> habitats) with translated paths
# Build Losgstash input data for ES from these input data :
# 1 - relations file
# 2 - BioNLP_onto,
# 3 - taxid_microorganisms
#####
def beyond_db_preprocessv2(relations,cols,BioNLP_onto,taxid_microorganisms):
#     relations = "/home/rodrique/Bureau/Jupyter-notebook/Beyond/data-examples/relations.full.tsv/relations_min.tsv"
    data = pd.read_csv(relations, delimiter='\t')
    df = data[cols]
    filtered_df = df[df['form_microorganism'].notnull()]
    
    filtered_df_dic = filtered_df.to_dict('records')
    val = list(map(dict, set(tuple(sorted(d.items())) for d in filtered_df_dic)))
    print(len(val))
    
    test = filtered_df[:5] # consider subset for test
    
    # define translated path --> ordered list of habitats
    test['translated_path_habitat'] = test['paths_habitat'].str.split("/")
        
    dc = {}
    synod = {}
    with open (BioNLP_onto, "r") as hfile:
        sp = hfile.read()
        lines = sp.split("\n\n")
        for line in lines:
            item = line.split('\n\n\n')
            items = [i.split('\n') for i in item]
            items = [x.split(': ') for x in items[0]]
            id_name = [l for l in items if l[0]=='id' or l[0]=='name']
            syno =  [l for l in items if l[0]=='id' or l[0]=='synonym']

            if len(id_name)>0:
                dc[id_name[0][1]] = id_name[1][1]
            #   print(syno)
            ########## We can add synonyms if needed
            if len(syno)>1:
                synod[syno[0][1]] = [ i[1] for i in syno[1:]]
            ############

    for k,v in dc.items():
        test['translated_path_habitat'] = test.apply(lambda x:[i.replace(k,v) for i in x['translated_path_habitat'] if i], axis=1)
        
    # pop current habitat from the path
    test['translated_path_habitat'] = test['translated_path_habitat'].apply(lambda x:x[:-1])
    
    
    ############Build translated path for each micro-organism
#     taxid_microorganisms = '/home/rodrique/Bureau/Jupyter-notebook/Beyond/data-examples/taxid_microorganisms.tsv/taxid_microorganisms.tsv'
    taxid_data = pd.read_csv(taxid_microorganisms, delimiter='\t')
    
    taxid_data = taxid_data[ ['taxid', 'taxon']]
    
    taxid_data_dic = taxid_data.set_index('taxid').T.to_dict('r')
    
    test['translated_paths_microorganism'] = test['paths_microorganism'].str.split("/")
    
    for k,v in taxid_data_dic[0].items():
        test['translated_paths_microorganism'] = test.apply(lambda x:[i.replace(k,v) for i in x['translated_paths_microorganism'] if i], axis=1)
    
    # pop current habitat from the path
    test['translated_paths_microorganism'] = test['translated_paths_microorganism'].apply(lambda x:x[:-1])

    test_dic = test.to_dict('records')    
    for d in test_dic:
        d['pmid'] = str(d['pmid'])
        
    ################### Save record!
    for mtd in test_dic:
        save_to_jsonl(mtd, 'beyond_db_uniq5.jsonl')

In [ ]:
Index(['taxid', 'form_microorganism','paths_microorganism',
       'obtid_mi', 'form_habitat',       'paths_habitat', 
       'offset_microorganism_mi', 'offset_habitat',
      'obtid_hbt', 'form_phenotype',
       'lemma_phenotype', 'canonical_phenotype', 'paths_phenotype',
       'offset_microorganism_hbt', 'offset_phenotype',
       'section_use', 'obtid_use', 'form_use', 'lemma_use', 'canonical_use',
       'paths_use', 'offset_microorganism_use', 'offset_use']

In [10]:
relations = "../data-examples/relations.full.tsv/relations_min.tsv"
full_relations = "/home/rodrique/Bureau/Jupyter-notebook/Beyond/data-examples/data/min/full_relations.csv"
BioNLP_onto = '../data-examples/BioNLP-OST+EnovFood.txt'
taxid_microorganisms = '../data-examples/taxid_microorganisms.tsv/taxid_microorganisms.tsv'

In [11]:
beyond_db_preprocess(relations,BioNLP_onto,taxid_microorganisms)

/home/rodrique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


100309


/home/rodrique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/rodrique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/rodrique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [ ]:
############ Additional data
# * relation exhibitsentre Taxon et Phenotype
# * relation studied for entre Taxon et Use
# Vous aviez déja eu la relation lives in entre Taxon et Habitat
##########